In [1]:
from gc_val import GC_Val
import pickle
import pandas as pd

In [2]:
val = GC_Val("data.csv", "neighborhoods.pkl")
threshold = 0.7
max_dist = 15 #km

In [12]:
file = open("false_responses.pkl", "rb")
false_resp = pickle.load(file)
file.close()
len(false_resp)

3533

In [13]:
# Separating the data into interested classifiers
high_conf = []
mid_conf = []
low_conf = []
addrLine_issue = []
admin_district = []
rooftop = []
interpol = []
interpol_offset = []
no_bldg = []
f_matches = {}

for ind in false_resp:
    # Getting the interested classifiers
    if (conf:=val.get_confidence(ind)) == 'High':
        high_conf.append(ind)
    elif (conf:=val.get_confidence(ind)) == 'Medium':
        mid_conf.append(ind)
    elif (conf:=val.get_confidence(ind)) == 'Low':
        low_conf.append(ind)
        
    calc_method = val.get_calculation_method(ind)
    # ['Rooftop', 'InterpolationOffset', 'Interpolation']

    if calc_method == 'Rooftop':
        rooftop.append(ind)
    elif calc_method == 'InterpolationOffset':
        interpol_offset.append(ind)
    elif calc_method == 'Interpolation':
        interpol.append(ind)
        
    match_codes = val.get_match_codes(ind)
    if str(match_codes) in f_matches.keys():
        f_matches[str(match_codes)] += 1
    else:
        f_matches[str(match_codes)] = 1
        
    # Finding out the reason for false entry
    if not val.has_addressLine(ind):
        addrLine_issue.append(ind)
        continue
    
    if not val.same_admin_district(ind):
        admin_district.append(ind)
        continue
        
    if not val.has_building_number(ind):
        no_bldg.append(ind)
print("Total faulty entries:", len(false_resp))    
print("High conf:", len(high_conf))
print("Medium conf:", len(mid_conf))
print("Low conf:", len(low_conf))
print()
print("AddressLine issue:", len(addrLine_issue))
print("Admin district issue:", len(admin_district))
print("Missing building number:", len(no_bldg))
print()
print("Rooftop method:", len(rooftop))
print("Interpolation method:", len(interpol))
print("Interpolation_offset method:", len(interpol_offset))
display(f_matches)

Total faulty entries: 3533
High conf: 3214
Medium conf: 287
Low conf: 32

AddressLine issue: 130
Admin district issue: 283
Missing building number: 105

Rooftop method: 3248
Interpolation method: 98
Interpolation_offset method: 187


{"['UpHierarchy']": 169,
 "['Ambiguous', 'UpHierarchy']": 24,
 "['Good']": 3286,
 "['Ambiguous']": 54}

In [14]:
# Finding total number of high, medium, and low confidence entries
high = 0
mid = 0
low = 0
r = 0
inter = 0
io = 0
combs = {}
for i in range(len(val.df)):
    try:
        if val.get_confidence(i) == "High":
            high += 1
        elif val.get_confidence(i) == "Medium":
            mid += 1
        elif val.get_confidence(i) == "Low":
            low += 1
        
        method = val.get_calculation_method(i)
        if method == 'Rooftop':
            r += 1
        elif method == 'Interpolation':
            inter += 1
        elif method == 'InterpolationOffset':
            io += 1

        match_codes = val.get_match_codes(i)
        if str(match_codes) in combs.keys():
            combs[str(match_codes)] += 1
        else:
            combs[str(match_codes)] = 1

    except Exception as e:
        continue
print("High confidence:", high)
print("Medium confidence:", mid)
print("Low confidence:", low)
print()
print("Rooftop:", r)
print("Interpolation:", inter)
print("InterpolationOffset:", io)
print()
display(combs)

High confidence: 96016
Medium confidence: 3627
Low confidence: 350

Rooftop: 96613
Interpolation: 992
InterpolationOffset: 2388



{"['Good']": 97066,
 "['UpHierarchy']": 1978,
 "['Ambiguous', 'UpHierarchy']": 189,
 "['Ambiguous']": 760}

In [15]:
# Statistics
digits = 3

# High conf:
print("Percentage of high-confidence faulty entries: " + str(round(len(high_conf)/high * 100, digits)) + "%")
print("Percentage of medium-confidence faulty entries: " + str(round(len(mid_conf)/mid * 100, digits)) + "%")
print("Percentage of low-confidence faulty entries: " + str(round(len(low_conf)/low * 100, digits)) + "%" )
print("Total percentage of faulty entries: " + str(round(len(false_resp) / len(val.df) * 100, digits)) + "%")
print()
print("Percentage of rooftop faulty entries: " + str(round(len(rooftop)/r * 100, digits)) + "%")
print("Percentage of interpolation faulty entries: " + str(round(len(interpol)/inter * 100, digits)) + "%")
print("Percentage of interpolation_offset faulty entries: " + str(round(len(interpol_offset)/io * 100, digits)) + "%")
print()
for key in f_matches:
    percentage = round(f_matches[key]/combs[key] * 100, digits)
    print(f"Percentage of faulty entries with match_code {key}: {percentage}%")

Percentage of high-confidence faulty entries: 3.347%
Percentage of medium-confidence faulty entries: 7.913%
Percentage of low-confidence faulty entries: 9.143%
Total percentage of faulty entries: 3.533%

Percentage of rooftop faulty entries: 3.362%
Percentage of interpolation faulty entries: 9.879%
Percentage of interpolation_offset faulty entries: 7.831%

Percentage of faulty entries with match_code ['UpHierarchy']: 8.544%
Percentage of faulty entries with match_code ['Ambiguous', 'UpHierarchy']: 12.698%
Percentage of faulty entries with match_code ['Good']: 3.385%
Percentage of faulty entries with match_code ['Ambiguous']: 7.105%
